In [1]:
import os
print(os.path.exists(".../dataset/dataset_train/"))

False


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

from myxtts.config.config import XTTSConfig,ModelConfig,DataConfig,TrainingConfig
from myxtts import get_xtts_model, get_trainer, get_inference_engine
from myxtts.utils.performance import start_performance_monitoring
start_performance_monitoring()

m=ModelConfig(
    text_encoder_dim=256,
    decoder_dim=512,
    n_mels=80,
    use_voice_conditioning=True
)
t=TrainingConfig(
    epochs=200,
    learning_rate=5e-5,
    optimizer="adamw",
    warmup_steps=2000,
)
d=DataConfig(
    batch_size=2,
    metadata_train_file="metadata_train.csv",
    metadata_eval_file="metadata_eval.csv",
    wavs_train_dir="wavs",
    wavs_eval_dir="wavs",
    sample_rate=22050,
    normalize_audio=True,
    num_workers=16,
    enable_memory_mapping=True
)
# Load configuration
config = XTTSConfig(
    model=m,
    training=t, 
    data=d
)
#config = XTTSConfig.from_yaml("config.yaml")



2025-09-13 02:14:08.989914: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757717049.007369 1151298 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757717049.013507 1151298 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1757717049.032174 1151298 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757717049.032190 1151298 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757717049.032192 1151298 computation_placer.cc:177] computation placer alr

Performance monitoring started


In [ ]:
# Training
model = get_xtts_model()(config.model)
trainer = get_trainer()(config, model)
train_dataset, val_dataset = trainer.prepare_datasets(train_data_path="../dataset/dataset_train",val_data_path="../dataset/dataset_eval")
trainer.train(train_dataset, val_dataset)


I0000 00:00:1757717053.429481 1151298 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22135 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9
I0000 00:00:1757717053.430090 1151298 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 20786 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:4d:00.0, compute capability: 8.9
2025-09-13 02:14:14,979 - MyXTTS - INFO - Using device: CPU
2025-09-13 02:14:14,980 - MyXTTS - INFO - Using strategy: _DefaultDistributionStrategy


GPU setup error: Physical devices cannot be modified after being initialized
Loaded 20509 items for train subset
Loaded 2591 items for val subset
Precomputing mel spectrograms to ../dataset/dataset_train/processed/mels_sr22050_n80_hop256 (overwrite=False)...
All mel spectrograms already cached.
Precomputing mel spectrograms to ../dataset/dataset_eval/processed/mels_sr22050_n80_hop256 (overwrite=False)...
All mel spectrograms already cached.


2025-09-13 02:14:22,731 - MyXTTS - INFO - Cache verify: train {'checked': 20509, 'fixed': 0, 'failed': 0}, val {'checked': 2591, 'fixed': 0, 'failed': 0}
2025-09-13 02:14:23,849 - MyXTTS - INFO - Using cached items - train: 20509, val: 2591
2025-09-13 02:14:25,316 - MyXTTS - INFO - Training samples: 20509
2025-09-13 02:14:25,317 - MyXTTS - INFO - Validation samples: 2591
2025-09-13 02:14:25,317 - MyXTTS - INFO - Data loading performance:
2025-09-13 02:14:25,317 - MyXTTS - INFO - === Data Loading Profile ===

Cache Efficiency: 0.0%
  Hits: 0
  Misses: 0
  Errors: 0


2025-09-13 02:14:25,318 - MyXTTS - INFO - Starting training for 200 epochs
2025-09-13 02:14:25,318 - MyXTTS - INFO - Current step: 0


Performance monitoring started


Epoch 0:   0%|          | 0/10255 [00:00<?, ?it/s]/home/dev371/.local/lib/python3.10/site-packages/keras/src/layers/layer.py:421: UserWarning: `build()` was called on layer 'xtts', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
Epoch 0:   0%|          | 38/10255 [01:17<4:53:44,  1.72s/it, loss=94.5190, step=38, data_ms=0.4, comp_ms=1675.3, mel=2.10, stop=0.071]  

In [ ]:

# Inference
inference = get_inference_engine()(config, checkpoint_path="./checkpoints/best")
result = inference.synthesize("Hello world!")
inference.save_audio(result["audio"], "output.wav")